In [1]:
#get_ipython().system('pip install pycryptodome==3.14.0')

In [2]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

In [3]:
# 2) RSA Crypto 클래스 구현 
class RSACrypto:
    def __init__(self):
        self.key_size = 1024  # 키 크기는 1024로 고정
        
    def Create_KeyPair(self):
        # RSA 키 쌍 생성
        key = RSA.generate(self.key_size)
        # 공개키와 개인키 추출
        public_key = key.publickey().exportKey()
        private_key = key.exportKey()
        return public_key, private_key
    
    def Encrypt(self, data, key):
        # 키 가져오기
        rsa_key = RSA.importKey(key)

        # 공개키가 아니면 에러 발생
        if not rsa_key.has_private():
            # 암호화 도구 초기화
            cipher = PKCS1_OAEP.new(rsa_key)
            # 데이터 암호화
            encrypted_data = cipher.encrypt(data.encode())
            return encrypted_data
        else:
            raise ValueError("Private key provided for encryption, expected public key.")
    
    def Decrypt(self, data, key):
        # 키 가져오기
        rsa_key = RSA.importKey(key)

        # 개인키가 아니면 에러 발생
        if rsa_key.has_private():
            # 암호화 도구 초기화
            cipher = PKCS1_OAEP.new(rsa_key)
            # 데이터 복호화
            decrypted_data = cipher.decrypt(data).decode()
            return decrypted_data
        else:
            raise ValueError("Public key provided for decryption, expected private key.")

## `exportKey` 및 `importKey` 사용의 필요성

RSA 키 객체는 복잡한 데이터 구조로 이루어져 있어, 실제 활용시에는 적절한 문자열 또는 바이트 형식으로의 변환이 필요합니다. 이를 위해 `exportKey`와 `importKey` 메소드를 사용합니다.

### 1. 포멧화 및 저장

RSA 키는 메모리 상의 복잡한 데이터 구조입니다. 파일로 저장하거나 네트워크를 통해 전송하기 위해서는 이 데이터 구조를 적절한 문자열 또는 바이트 형식으로 변환해야 합니다. `exportKey`는 이런 변환 작업을 수행, 키를 다양한 포맷으로 출력해줍니다.

### 2. 표준화

`exportKey`를 통해 키는 표준 포맷, 예를 들어 **PEM** 또는 **DER**와 같은 포맷으로 출력됩니다. 이러한 표준 포맷은 다양한 암호화 도구나 솔루션 간의 호환성을 보장하는 데 중요합니다.

### 3. 키 교환

암호화 통신에서 키 교환은 필수적입니다. 키 객체를 직접 전송하는 것은 실질적으로 불가능하므로, 문자열 또는 바이트 형식으로 변환된 키를 전달해야 합니다.

### 4. 복원 및 사용

`importKey`는 `exportKey`를 통해 변환된 키 데이터를 다시 RSA 키 객체로 복원합니다. 즉, 실제 암호화나 복호화 작업에 사용하기 위해 변환 작업을 수행합니다.

### 5. 보안

키를 안전하게 저장하거나 전송하기 위해 추가적인 암호화 방법을 적용할 수 있습니다. 이때, 문자열 또는 바이트 형식의 키 데이터를 기반으로 이러한 작업을 수행하기 쉽습니다.

---
# ✅**RSA Test**

In [4]:
# plaintext는 사용자로부터 string 형태로 받아옴
plaintext = input("평문을 입력하세요: ")

rsa = RSACrypto()
public_key, private_key = rsa.Create_KeyPair()

encrypted_data = rsa.Encrypt(plaintext , public_key)
#print("암호화 결과 :", encrypted_data)

decrypted_data = rsa.Decrypt(encrypted_data, private_key)
# 복호화된 텍스트가 비어있는지 확인하고, 그에 따라 적절한 메시지를 출력
if not decrypted_data.strip():  # strip()을 사용하여 공백만 있는 문자열도 체크
    print("복호화 결과 :", "빈 평문입니다.")
else:
    print("복호화 결과 :", decrypted_data)

평문을 입력하세요:  Hello, World!


복호화 결과 : Hello, World!


---
# **RSA Auto-test**

In [5]:
# RSA 암호화/복호화
# 정상적인 공개키 및 개인키를 사용하여 RSA 암호화 및 복호화가 올바르게 작동함
def test_rsa_encryption_decryption():
    print("\n[+] Testing RSA encryption/decryption")

    plaintext = "Test data for RSA encryption"
    rsa = RSACrypto()

    public_key, private_key = rsa.Create_KeyPair()
    encrypted_data = rsa.Encrypt(plaintext, public_key)
    decrypted_data = rsa.Decrypt(encrypted_data, private_key)

    assert decrypted_data == plaintext
    print("RSA encryption/decryption: PASSED") # 정상적인 결과

# 긴 문자열에 대해서도 RSA가 올바르게 작동함
def test_rsa_with_long_string():
    print("\n[+] Testing RSA with long string")

    plaintext = "A" * 86  # Modify this line 100바이트의 문자열은 안됨
    rsa = RSACrypto()

    public_key, private_key = rsa.Create_KeyPair()
    encrypted_data = rsa.Encrypt(plaintext, public_key)
    decrypted_data = rsa.Decrypt(encrypted_data, private_key)

    assert decrypted_data == plaintext
    print("RSA with long string: PASSED") # 정상적인 결과
    
# 빈 평문으로 RSA 테스트: 평문이 없을 때도 RSA 올바르게 작동함
def test_rsa_empty_plaintext():
    print("\n[+] Testing RSA with empty plaintext")

    plaintext = ""
    rsa = RSACrypto()

    public_key, private_key = rsa.Create_KeyPair()
    encrypted_data = rsa.Encrypt(plaintext, public_key)
    decrypted_data = rsa.Decrypt(encrypted_data, private_key)

    assert decrypted_data == plaintext
    print("RSA with empty plaintext: PASSED") # 정상적인 결과

# RSA에서 개인 키로 암호화를 수행하는 것은 허용되지 않음, 따라서 테스트가 실패하는 것은 올바른 동작임
def test_rsa_encryption_with_private_key():
    print("\n[+] Testing RSA encryption using private key (--> which is not supposed to be done)")

    plaintext = "Test data for RSA encryption"
    rsa = RSACrypto()

    public_key, private_key = rsa.Create_KeyPair()
    try:
        encrypted_data = rsa.Encrypt(plaintext, private_key)
        print("RSA encryption using private key: UNEXPECTED PASSED") # 비정상적인 결과
    except Exception:
        print("RSA encryption using private key: EXPECTED FAILED") # 정상적인 결과

# RSA에서는 공개 키로 복호화를 시도하는 것은 허용되지 않음,. 따라서 테스트가 실패하는 것은 올바른 동작임
def test_rsa_decryption_with_public_key():
    print("\n[+] Testing RSA decryption using public key (--> which is not supposed to be done)")

    plaintext = "Test data for RSA encryption"
    rsa = RSACrypto()

    public_key, private_key = rsa.Create_KeyPair()
    encrypted_data = rsa.Encrypt(plaintext, public_key)
    try:
        decrypted_data = rsa.Decrypt(encrypted_data, public_key)
        print("RSA decryption using public key: UNEXPECTED PASSED") # 비정상적인 결과
    except Exception:
        print("RSA decryption using public key: EXPECTED FAILED") # 정상적인 결과



In [6]:
test_rsa_encryption_decryption()
test_rsa_with_long_string()
test_rsa_empty_plaintext()
test_rsa_encryption_with_private_key()
test_rsa_decryption_with_public_key()


[+] Testing RSA encryption/decryption
RSA encryption/decryption: PASSED

[+] Testing RSA with long string
RSA with long string: PASSED

[+] Testing RSA with empty plaintext
RSA with empty plaintext: PASSED

[+] Testing RSA encryption using private key (--> which is not supposed to be done)
RSA encryption using private key: EXPECTED FAILED

[+] Testing RSA decryption using public key (--> which is not supposed to be done)
RSA decryption using public key: EXPECTED FAILED
